### **PREPROCESSING & CLEANING DATA**

In [2]:
import pandas as pd
import numpy as np

In [3]:
# 1. Đọc dữ liệu
data = pd.read_csv('Data/lung_disease_data.csv')  # Đảm bảo đường dẫn chính xác

# In thông tin cơ bản (giữ lại phần này để bạn kiểm tra)
print("Thông tin dữ liệu ban đầu:\n")
print(data.info())
print("\n5 dòng dữ liệu đầu tiên:\n")
print(data.head())
print("\nSố lượng giá trị thiếu trên mỗi cột:\n")
print(data.isnull().sum())
print("\n--------------------------------------------------------\n")


Thông tin dữ liệu ban đầu:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              4900 non-null   float64
 1   Gender           4900 non-null   object 
 2   Smoking Status   4900 non-null   object 
 3   Lung Capacity    4900 non-null   float64
 4   Disease Type     4900 non-null   object 
 5   Treatment Type   4900 non-null   object 
 6   Hospital Visits  4900 non-null   float64
 7   Recovered        4900 non-null   object 
dtypes: float64(3), object(5)
memory usage: 325.1+ KB
None

5 dòng dữ liệu đầu tiên:

    Age  Gender Smoking Status  Lung Capacity Disease Type Treatment Type  \
0  71.0  Female             No           4.49         COPD        Therapy   
1  34.0  Female            Yes            NaN   Bronchitis        Surgery   
2  80.0    Male            Yes           1.95         COPD            NaN   
3  40.0  Fem

In [4]:
# 2. Data Cleaning

# 2.1 Xử lý dữ liệu bị thiếu

# Hàm điền giá trị thiếu cho cột số bằng median
def impute_numerical(df, column):
    median_val = df[column].median()
    df[column] = df[column].fillna(median_val)  # Gán kết quả trở lại cột

def impute_categorical(df, column):
    mode_val = df[column].mode()[0]
    df[column] = df[column].fillna(mode_val)  # Gán kết quả trở lại cột

# Áp dụng cho các cột tương ứng
numerical_cols = ['Lung Capacity', 'Hospital Visits']
categorical_cols = ['Gender', 'Smoking Status', 'Disease Type', 'Treatment Type']

for col in numerical_cols:
    impute_numerical(data, col)

for col in categorical_cols:
    impute_categorical(data, col)

# 2.2 Xử lý dữ liệu trùng lặp
data.drop_duplicates(inplace=True)
print(f"Số lượng dòng sau khi xóa trùng lặp: {len(data)}")

# 2.3 Chuẩn hóa dữ liệu category
def standardize_text(df, column):
    df[column] = df[column].str.strip().str.lower() #Loại bỏ khoảng trắng đầu cuối và viết thường
    return df

for col in categorical_cols:
    data = standardize_text(data, col)

print("\nSố lượng giá trị thiếu trên mỗi cột (sau xử lý missing):\n")
print(data.isnull().sum())
print("\n--------------------------------------------------------\n")

Số lượng dòng sau khi xóa trùng lặp: 5094

Số lượng giá trị thiếu trên mỗi cột (sau xử lý missing):

Age                300
Gender               0
Smoking Status       0
Lung Capacity        0
Disease Type         0
Treatment Type       0
Hospital Visits      0
Recovered          300
dtype: int64

--------------------------------------------------------



In [5]:
# 3. Encoding biến phân loại (One-Hot Encoding)
def one_hot_encode(df, column):
    dummies = pd.get_dummies(df[column], prefix=column, dummy_na=False)  # dummy_na=False để không tạo cột cho NaN
    df = pd.concat([df, dummies], axis=1)
    df.drop(column, axis=1, inplace=True)
    return df

for col in categorical_cols:
    data = one_hot_encode(data, col)
    
# Loại bỏ cột đầu tiên của one-hot encoding
data.drop(data.filter(regex='_no$').columns, axis=1, inplace=True)

In [6]:
# 4. Encoding biến mục tiêu (Recovered)
data['Recovered'] = data['Recovered'].map({'yes': 1, 'no': 0}).fillna(0) # Điền NaN bằng 0

In [7]:
# 5. Scaling/Normalization (Min-Max Scaling)
def min_max_scale(df, column):
    min_val = df[column].min()
    max_val = df[column].max()
    df[column] = (df[column] - min_val) / (max_val - min_val)

for col in numerical_cols:
    min_max_scale(data, col)


In [8]:
# 6. Loại bỏ cột đầu tiên của one-hot encoding (tránh đa cộng tuyến)
data.drop(data.filter(regex='_no$').columns, axis=1, inplace=True)


In [9]:
# 7. Chuyển đổi kiểu dữ liệu Boolean sang Integer
for col in data.columns:
    if data[col].dtype == 'bool':
        data[col] = data[col].astype(int)


In [10]:
# 8. Xuất ra file CSV
data.to_csv('lung_disease_data_preprocessed.csv', index=False) # Lưu vào file mới
print("Đã xuất file 'lung_disease_data_preprocessed.csv' vào thư mục Data.")

Đã xuất file 'lung_disease_data_preprocessed.csv' vào thư mục Data.
